In [1]:
import polars as pl
from tqdm import tqdm

In [2]:

offers_path = "../data/offer.csv"
hotels_path = "../data/hotels.csv"
output_path = "../data/offers.parquet"

offers = pl.read_csv(offers_path)
hotels = pl.read_csv(hotels_path)

df = offers.join(hotels, on="hotelid", how="left")
    
datetime_columns = [
        "outbounddeparturedatetime",
        "inbounddeparturedatetime",
        "outboundarrivaldatetime",
        "inboundarrivaldatetime"
]

for col in tqdm(datetime_columns, desc="Parsing datetime columns"):
    df = df.with_columns(
        pl.col(col).str.strptime(pl.Datetime, fmt="%Y-%m-%d %H:%M:%S", strict=False)
)

df = df.with_columns([
    pl.col("outbounddeparturedatetime").dt.date().alias("outbound_date"),
    (pl.col("inbounddeparturedatetime") - pl.col("outbounddeparturedatetime")).dt.days().cast(pl.UInt8).alias("duration")
])

#Datatypes for ClickHouse
columns_to_cast = {
    "hotelstars": pl.Float32,
    "price": pl.Float32,
    "oceanview": pl.UInt8,
    "countadults": pl.UInt8,
    "countchildren": pl.UInt8,
    "roomtype": pl.Categorical,
    "mealtype": pl.Categorical,
    "outbounddepartureairport": pl.Categorical,
    "lat": pl.Float64,
    "lon": pl.Float64,
    }

for col, dtype in tqdm(columns_to_cast.items(), desc="Casting columns"):
    df = df.with_columns([
        pl.col(col).cast(dtype)
])

df.write_parquet(output_path)
print("Saved Parquet:", output_path)

FileNotFoundError: No such file or directory (os error 2): ../data/offers.csv